In [1]:
import datetime
import rasterio
import geopandas
import glob
import fiona
import geojson
import shapely.geometry
import os
import shutil
import numpy as np
import sys
import logging
import random
import time

from pathlib import Path
from sentinelsat import read_geojson
from datetime import timedelta
from rasterio.plot import show
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union
from shapely.geometry import Polygon
from geojson import FeatureCollection,dump
from rasterio.mask import mask
from osgeo import gdal

In [2]:
## Clip methods
def deleteFolder(path):
    try:
        shutil.rmtree(path)
    except OSError as e:
        print ("Error: %s - %s." % (e.filename, e.strerror))

def showSatImage(path):
        src = rasterio.open(path)
        titlestring = str(src.crs.data['init'])
        show(src,transform=src.transform,title=titlestring,with_bounds=True)
        print('Height ' +str(src.height))
        print('Width ' + str(src.width))
        print('Spatial res ' + str(src.res[0]) +','+str(src.res[1]))
        print('Area ' + str(src.bounds))

def clip_optic_image(baseInPath,baseOutPath,aoi,bandName,band='',transform=None,height=None,width=None):

        with rasterio.open(baseInPath + band) as img:
            #outbool,outtransform,outwindow = rasterio.mask.raster_geometry_mask(img, aoi,crop=True)
            out_img, out_transform= mask(img, aoi,crop=True)

        # use the metadata from our original mosaic
        meta = img.meta.copy()
        if transform!=None:
            out_transform=transform

        # update metadata with new, clipped mosaic's boundaries
        meta.update({"driver": "GTiff",
                    "transform": out_transform,
                     "height": height,
                     "width": width})
        end_path = baseOutPath + '//'+bandName+'.tif'
        # write the clipped-and-cropped dataset to a new GeoTIFF
        with rasterio.open(end_path, 'w', **meta) as dst:
            dst.write(out_img)
        return end_path
        
def clip_image(baseInPath,baseOutPath,aoi,bandName,band=''):

        with rasterio.open(baseInPath + band) as img:
            #outbool,outtransform,outwindow = rasterio.mask.raster_geometry_mask(img, aoi,crop=True)
            out_img, out_transform= mask(img, aoi,crop=True)

        # use the metadata from our original mosaic
        meta = img.meta.copy()

        # update metadata with new, clipped mosaic's boundaries
        meta.update({"driver": "GTiff",
                    "transform": out_transform,
                     "height": out_img.shape[1],
                     "width": out_img.shape[2]})
        end_path = baseOutPath + '//'+bandName+'.tif'
        # write the clipped-and-cropped dataset to a new GeoTIFF
        with rasterio.open(end_path, 'w', **meta) as dst:
            dst.write(out_img)
        return out_transform,end_path, out_img.shape[1],out_img.shape[2]
def cog_image(baseInPath,baseOutPath):
    if baseInPath == baseOutPath:
        baseOutPath=baseOutPath.replace('.tif','')
        baseOutPath=baseOutPath+'_cog.tif'        
    !gdal_translate {baseInPath} {baseOutPath} -of COG -co COMPRESS=DEFLATE -co NUM_THREADS=ALL_CPUS -co BIGTIFF=YES --config GDAL_CACHEMAX 16384
    
def create_cog(img, factors=[2, 4, 8, 16, 32], compression='DEFLATE'):
    # Define a new configuration, save the previous configuration,
    # and then apply the new one.
    new_config = {
        'GDAL_CACHEMAX': '8192',
        'COMPRESS_OVERVIEW': 'NONE'  # Compression will be made in the next step
    }
    prev_config = {
        key: gdal.GetConfigOption(key) for key in new_config.keys()}
    for key, val in new_config.items():
        gdal.SetConfigOption(key, val)

    InputImage = str(img)
    Image = gdal.Open(InputImage, 1)  # 0 = read-only, 1 = read-write.
    Image.BuildOverviews("NEAREST", factors)
    del Image

    # Restore previous configuration.
    for key, val in prev_config.items():
        gdal.SetConfigOption(key, val)


    # Run gdal_translate to properly tile and compress gtiff file after overviews have been added
    # Note: Should not be necessary, but the current version of gdal destroys tiles when adding overviews
    img_tmp = Path(img.parent, 'temp' + img.suffix)
    img.rename(img_tmp)

    ds = gdal.Open(str(img_tmp))  # img is a Path object but gdal requires a string, therefore str(img)
    # NOTE: ZSTD and WEBP compression is not working here
    # (https://lists.osgeo.org/pipermail/gdal-dev/2018-November/049289.html)
    if compression == 'NONE':
        creation_options = ['TILED=YES', 'COPY_SRC_OVERVIEWS=YES', 'BLOCKXSIZE=512', 'BLOCKYSIZE=512',
                            'NUM_THREADS=ALL_CPUS', 'BIGTIFF=IF_SAFER']
    elif compression == 'DEFLATE':
        creation_options = ['COMPRESS=DEFLATE', 'TILED=YES', 'COPY_SRC_OVERVIEWS=YES',
                            'BLOCKXSIZE=512', 'BLOCKYSIZE=512', 'NUM_THREADS=ALL_CPUS', 'BIGTIFF=YES']
    elif compression == 'JPEG':
        creation_options = ['COMPRESS=JPEG', 'PHOTOMETRIC=YCBCR', 'JPEG_QUALITY=90', 'TILED=YES',
                            'COPY_SRC_OVERVIEWS=YES', 'BLOCKXSIZE=512', 'BLOCKYSIZE=512', 'NUM_THREADS=ALL_CPUS',
                            'BIGTIFF=IF_SAFER']
    config_options = ['GDAL_CACHEMAX=8192']
    translate_options = gdal.TranslateOptions(creationOptions=creation_options)
    ds = gdal.Translate(str(img), ds, options=translate_options, config_options=config_options)
    ds = None

    os.remove(img_tmp)

In [3]:
def downloadBiome(biome,outputPath,logger):
    process_files = True
    ## Using same data_dir
    biome_geojson = str(biome)
    data_dir = Path.cwd() / 'data'
    current_order_id = 'test_order_' + datetime.datetime.today().strftime('%Y%m%d-%H%M%S')
    
    ## LTA down to only 30 days so using multiple accounts to trigger download. 
    userTuple = [['pandagud','damp4ever'],['pandagud2','damp4ever'],['pandagud3','damp4ever'],['au524478','Palantir1234']]
    current_user = random.choice(userTuple)
    print(current_user[0])
    ## Downloading Sentinel-2
    !python app.py --username {current_user[0]} --password {current_user[1]} --order_id {current_order_id} --data_directory data --geojson {biome_geojson} --satellite s2 --startdate 20200801 --enddate 20210501 --s2_num_proc 4 --s2_relative_orbit 0 --s2_max_cloudcoverage 1
    ## based on Sentinel-2 geolocations we are downloading Sentinel-1
    geojson_path = (data_dir / 'orders' / current_order_id).with_suffix('.geojson')
    if not os.path.exists(geojson_path):
        logger.info('################################################')
        logger.info('No sentinel-2 product for this biome')
        return
    S2_geojson = read_geojson(geojson_path)
    start_S1_date = S2_geojson.get('features')[0].get('properties').get('ingestiondate')
    start_S1_date = start_S1_date.split('T')[0]
    start_S1_date = datetime.datetime.strptime(start_S1_date, '%Y-%m-%d').date()
    ## New end date for S1
    end_S1_date = start_S1_date + timedelta(days=7)
    start_S1_date = start_S1_date - timedelta(days=7)

    start_S1_date_str = str(start_S1_date).replace('-','')
    end_S1_date_str = str(end_S1_date).replace('-', '')

    current_order_id_SAR='test_order_SAR' + datetime.datetime.today().strftime('%Y%m%d-%H%M%S')
    
    current_user = random.choice(userTuple)
    
    ## Downloading Sentinel-1
    !python app.py --username {current_user[0]} --password {current_user[1]} --order_id {current_order_id_SAR} --s2_order_id {current_order_id} --data_directory data --geojson {geojson_path} --satellite s1 --startdate {start_S1_date_str} --enddate {end_S1_date_str} --s1_num_proc 4 --s1_del_intermediate True --s1_output_crs EPSG:3857 - Projected
    
    logger.info('#############')
    ## Arosic med Sentinel-1 som ref
    if process_files:
        logger.info('Done downloading and preprocess - will clip now')
        logger.info('#############')
        ## Sentinel-2 pathing
        data = geopandas.read_file(geojson_path)

        product = data['title']
        product = product.values[0]
        Sentinel2_name = str(product)
        utm_code = product[39:41]  # e.g. 10
        latitude_band = product[41:42]  # e.g. S
        square = product[42:44]  # e.g. DG
        tile_path = data_dir / 'output_data' / 's2' / 'tiles' / utm_code / latitude_band / square
        product_path = (tile_path / product).with_suffix('.SAFE')
        granule_path = product_path / 'GRANULE'
        if not os.path.exists(granule_path):
            logger.info('################################################')
            logger.info('No uncurrpoted sentinel-2 product for this biome')
            return
        files = [f for f in sorted(os.listdir(granule_path))]
        granule_path = granule_path / str(files[0])
        granule_path = granule_path / 'IMG_DATA'
        ## Only getting R10m atm 
        granule_path_10m = granule_path /'R10m'
        bands_path_10m = []
        for band in glob.glob(str(granule_path_10m) + '/*B0*'):
            bands_path_10m.append(os.path.basename(band))
        bands_path_10m.sort()

        baseInOpticPath = granule_path_10m
        
        ## Sentinel-1 pathing
        baseInSarPath = (data_dir / 'output_data' / 's1' /'combined')
        combined_there = False
        for combinedSar in glob.glob(str(baseInSarPath) + '/*.tiaf'):
            combined_there = True
            baseInSarPath = str(combinedSar) ## Quick fix but there should only be one
        if combined_there==False:
            logger.info('################################################')
            logger.info('No sentinel-1 product for this biome')
            return
        Sentinel1_name = Path(baseInSarPath).stem
        
        ## This is the output dir
        baseOutPath = outputPath
        output_path_SAR = baseOutPath + '/Sentinel1'
        if not os.path.exists(output_path_SAR):
            os.makedirs(output_path_SAR)
        output_path_OPTIC = baseOutPath +'/Sentinel2'
        if not os.path.exists(output_path_OPTIC):
            os.makedirs(output_path_OPTIC)
        band2 = '/'+str(bands_path_10m[0])
        band3 = '/'+str(bands_path_10m[1])
        band4 = '/'+str(bands_path_10m[2])
        band8 = '/'+str(bands_path_10m[3])

        ## Clipping Sentinel-1 and 2 based on their intersection Polygon. Storing this as a geojson 
        src = rasterio.open(str(baseInSarPath))
        sar_polygon = shapely.geometry.box(*src.bounds,ccw=True)
        src = rasterio.open(str(baseInOpticPath) + band2)
        optic_polygon = shapely.geometry.box(*src.bounds, ccw=True)
        intersection_poly = optic_polygon.intersection(sar_polygon)
        ## Storing intersection geojson. We need to disc. if we want three geojson - one for Sentinel-1, 2 and intersection.
        geom_in_geojson = []
        geom_in_geojson.append(geojson.Feature(geometry=intersection_poly, properties={}))
        feature_collection = FeatureCollection(geom_in_geojson)
        pathToFile = str(baseOutPath) + '//intersection.geojson'
        with open(pathToFile, 'w') as f:
            dump(feature_collection, f)
        geo_df = geopandas.GeoDataFrame.from_file(pathToFile)
        geo_df = geo_df.geometry.scale(xfact=0.95, yfact=0.95) ## scaling down
        geo_df.crs='EPSG:3857'
        intersections_panda = geo_df.geometry[0]
        geo_df.to_file(pathToFile,driver='GeoJSON')
        with fiona.open(pathToFile) as f:
            aoi = [feature["geometry"] for feature in f]
        list_cog_images = []
        logger.info("CLIPPING and COG")
        sen1_transform ,sar_clipped_path, height, width = clip_image(str(baseInSarPath),str(output_path_SAR),aoi,str(Sentinel1_name) + 'sar')
        list_cog_images.append(sar_clipped_path)
        band2_clipped_path = clip_optic_image(str(baseInOpticPath),str(output_path_OPTIC),aoi,str(Sentinel2_name)+'_B02',band=band2,transform=sen1_transform,height=height,width=width)
        list_cog_images.append(band2_clipped_path)

        band3_clipped_path=clip_optic_image(str(baseInOpticPath),str(output_path_OPTIC), aoi, str(Sentinel2_name)+'_B03',band=band3,transform=sen1_transform,height=height,width=width)
        list_cog_images.append(band3_clipped_path)

        band4_clipped_path=clip_optic_image(str(baseInOpticPath),str(output_path_OPTIC), aoi,str(Sentinel2_name)+ '_B04',band=band4,transform=sen1_transform,height=height,width=width)
        list_cog_images.append(band4_clipped_path)

        band8_clipped_path=clip_optic_image(str(baseInOpticPath),str(output_path_OPTIC), aoi,str(Sentinel2_name)+ '_B08',band=band8,transform=sen1_transform,height=height,width=width)
        list_cog_images.append(band8_clipped_path)

        ##TESTING 
        #showSatImage(sar_clipped_path)
        #showSatImage(band2_clipped_path)
        ##TESTING 
        for i in list_cog_images:
            print(str(i))
            create_cog(Path(i))



In [4]:
logname = '/workspace/data_landset8/ForestTestNoDebug'
logging.basicConfig(filename=logname + '.log',
                    filemode='a',
                    format='%(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger(logname)

In [12]:
inputPath = '/workspace/data_landset8/unzipped/Forest/BC/LC80200462014005LGN00/LC80200462014005LGN00.geojson'
output = '/workspace/data_landset8/unzipped/Forest/BC/LC80200462014005LGN00/'
downloadBiome(inputPath,output,logger)

pandagud
/workspace/DSD_paper_2020/SentinelProc/app/src/sentinelprocessors/s2processor.py:202: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  elif coreg_info is not 'coreg_error':  # Apply to other modes
/workspace/DSD_paper_2020/SentinelProc/app/data
I0512 10:59:40.231069 139681409169216 app.py:38] True
test_order_20210512-105932
I0512 10:59:40.231303 139681409169216 app.py:46] ####################################
I0512 10:59:40.231395 139681409169216 app.py:47] # Initializing Sentinel downloader #
I0512 10:59:40.231477 139681409169216 app.py:48] ####################################
I0512 10:59:40.231556 139681409169216 app.py:49] Order id: test_order_20210512-105932
I0512 10:59:40.231908 139681409169216 app.py:54] /workspace/data_landset8/unzipped/Forest/BC/LC80200462014005LGN00/LC80200462014005LGN00.geojson
I0512 10:59:40.585815 139681409169216 sentinel.py:209] Found 0 products
I0512 10:59:40.882905 139681409169216 sentinel.py:209] Found 8 products
I0512 10:59:40.88773

In [16]:
from arosics import COREG
import arosics

In [17]:
arosics.__version__

'1.3.0'